In [1]:
import pandas as pd
import numpy as np
import warnings
# Disable warnings
warnings.filterwarnings("ignore")

## Extracting features of 2020 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [3]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [4]:
df

Opening  Opening.1                  Title  \
0      J A N U A R Y          3             The Grudge   
1      J A N U A R Y         10             Underwater   
2      J A N U A R Y         10            Like a Boss   
3      J A N U A R Y         10          Three Christs   
4      J A N U A R Y         10      Inherit the Viper   
..               ...        ...                    ...   
270  D E C E M B E R         25       We Can Be Heroes   
271  D E C E M B E R         25      News of the World   
272  D E C E M B E R         25  One Night in Miami...   
273  D E C E M B E R         25  Promising Young Woman   
274  D E C E M B E R         30      Pieces of a Woman   

                                    Production company  \
0    Screen Gems / Stage 6 Films / Ghost House Pict...   
1    20th Century Fox / TSG Entertainment / Chernin...   
2                   Paramount Pictures / Artists First   
3                                            IFC Films   
4    Lionsgate / Barry Films / Tycor International ...   
..                                                 ...   
270  Netflix / Troublemaker Studios / Double R Prod...   
271  Universal Pictures / Playtone / Perfect World ...   
272                                     Amazon Studios   
273          Focus Features / FilmNation Entertainment   
274                             Netflix / Bron Studios   

                                         Cast and crew   Ref.  
0    Nicolas Pesce (director/screenplay); Andrea Ri...    [2]  
1    William Eubank (director); Brian Duffield, Ada...    [3]  
2    Miguel Arteta (director); Sam Pitman, Adam Col...    [4]  
3    Jon Avnet (director/screenplay); Eric Nazarian...    NaN  
4    Anthony Jerjen (director); Andrew Crabtree (sc...    [5]  
..                                                 ...    ...  
270  Robert Rodriguez (director/screenplay); Priyan...  [243]  
271  Paul Greengrass (director/screenplay); Luke Da...  [244]  
272  Regina King (director); Kemp Powers (screenpla...  [245]  
273  Emerald Fennell (director/screenplay); Carey M...  [246]  
274  Kornél Mundruczó (director); Kata Wéber (scree...  [247]  

[275 rows x 6 columns]

In [5]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'bd9d93e4649f8d7b42731beda117f8c6'
from tmdbv3api import Movie
tmdb_movie = Movie()
from tqdm import tqdm
tqdm.pandas()

In [6]:
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
        return np.NaN
    else:
        movie_id = result[0].id
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, tmdb.api_key))
        data_json = response.json()
        if data_json['genres']:
            genre_str = " "
            for i in range(len(data_json['genres'])):
                genres.append(data_json['genres'][i]['name'])
            return genre_str.join(genres)
        else:
            return np.NaN

# Modify the function to print the index and title, and handle the error
def safe_get_genre(title, index):
    try:
        return get_genre(title)
    except Exception as e:
        print(f"Error processing index: {index}")
        print(f"Title: {title}")
        print(f"Error: {e}")
        return np.NaN

# Apply the function with progress bar
df['genres'] = df.progress_apply(lambda row: safe_get_genre(str(row['Title']), row.name), axis=1)

 21%|████████████████▊                                                                | 57/275 [01:10<03:59,  1.10s/it]

Error processing index: 55
Title: No films had been released on this date as resulting the closure of theaters caused by the COVID-19 pandemic
Error: attribute name must be string, not 'int'


 47%|█████████████████████████████████████▊                                          | 130/275 [02:45<02:42,  1.12s/it]

Error processing index: 128
Title: The SpongeBob Movie: Sponge on the Run (Canadian theatrical release)
Error: attribute name must be string, not 'int'


100%|████████████████████████████████████████████████████████████████████████████████| 275/275 [05:58<00:00,  1.31s/it]


In [7]:
def get_id(x):
    result = tmdb_movie.search(x)
    if not result:
        return np.NaN
    else:
        movie_id = result[0].id
        return movie_id

# Modify the function to print the index and title, and handle the error
def safe_get_id(title, index):
    try:
        return get_id(title)
    except Exception as e:
        print(f"Error processing index: {index}")
        print(f"Title: {title}")
        print(f"Error: {e}")
        return np.NaN

# Apply the function with progress bar
df['id'] = df.progress_apply(lambda row: safe_get_id(str(row['Title']), row.name), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████| 275/275 [00:00<00:00, 3141.17it/s]

Error processing index: 55
Title: No films had been released on this date as resulting the closure of theaters caused by the COVID-19 pandemic
Error: attribute name must be string, not 'int'
Error processing index: 128
Title: The SpongeBob Movie: Sponge on the Run (Canadian theatrical release)
Error: attribute name must be string, not 'int'


In [8]:
df

Opening  Opening.1                  Title  \
0      J A N U A R Y          3             The Grudge   
1      J A N U A R Y         10             Underwater   
2      J A N U A R Y         10            Like a Boss   
3      J A N U A R Y         10          Three Christs   
4      J A N U A R Y         10      Inherit the Viper   
..               ...        ...                    ...   
270  D E C E M B E R         25       We Can Be Heroes   
271  D E C E M B E R         25      News of the World   
272  D E C E M B E R         25  One Night in Miami...   
273  D E C E M B E R         25  Promising Young Woman   
274  D E C E M B E R         30      Pieces of a Woman   

                                    Production company  \
0    Screen Gems / Stage 6 Films / Ghost House Pict...   
1    20th Century Fox / TSG Entertainment / Chernin...   
2                   Paramount Pictures / Artists First   
3                                            IFC Films   
4    Lionsgate / Barry Films / Tycor International ...   
..                                                 ...   
270  Netflix / Troublemaker Studios / Double R Prod...   
271  Universal Pictures / Playtone / Perfect World ...   
272                                     Amazon Studios   
273          Focus Features / FilmNation Entertainment   
274                             Netflix / Bron Studios   

                                         Cast and crew   Ref.  \
0    Nicolas Pesce (director/screenplay); Andrea Ri...    [2]   
1    William Eubank (director); Brian Duffield, Ada...    [3]   
2    Miguel Arteta (director); Sam Pitman, Adam Col...    [4]   
3    Jon Avnet (director/screenplay); Eric Nazarian...    NaN   
4    Anthony Jerjen (director); Andrew Crabtree (sc...    [5]   
..                                                 ...    ...   
270  Robert Rodriguez (director/screenplay); Priyan...  [243]   
271  Paul Greengrass (director/screenplay); Luke Da...  [244]   
272  Regina King (director); Kemp Powers (screenpla...  [245]   
273  Emerald Fennell (director/screenplay); Carey M...  [246]   
274  Kornél Mundruczó (director); Kata Wéber (scree...  [247]   

                                      genres        id  
0                    Horror Mystery Thriller    1970.0  
1    Horror Science Fiction Action Adventure  443791.0  
2                                     Comedy  526019.0  
3                                      Drama  418533.0  
4                       Crime Thriller Drama  634904.0  
..                                       ...       ...  
270             Family Action Fantasy Comedy  615677.0  
271                  Drama Western Adventure  581032.0  
272                                    Drama  661914.0  
273                     Thriller Crime Drama  582014.0  
274                                    Drama  641662.0  

[275 rows x 8 columns]

In [9]:
df_2020 = df[['Title','Cast and crew','genres','id']]

In [10]:
df_2020

Title                                      Cast and crew  \
0               The Grudge  Nicolas Pesce (director/screenplay); Andrea Ri...   
1               Underwater  William Eubank (director); Brian Duffield, Ada...   
2              Like a Boss  Miguel Arteta (director); Sam Pitman, Adam Col...   
3            Three Christs  Jon Avnet (director/screenplay); Eric Nazarian...   
4        Inherit the Viper  Anthony Jerjen (director); Andrew Crabtree (sc...   
..                     ...                                                ...   
270       We Can Be Heroes  Robert Rodriguez (director/screenplay); Priyan...   
271      News of the World  Paul Greengrass (director/screenplay); Luke Da...   
272  One Night in Miami...  Regina King (director); Kemp Powers (screenpla...   
273  Promising Young Woman  Emerald Fennell (director/screenplay); Carey M...   
274      Pieces of a Woman  Kornél Mundruczó (director); Kata Wéber (scree...   

                                      genres        id  
0                    Horror Mystery Thriller    1970.0  
1    Horror Science Fiction Action Adventure  443791.0  
2                                     Comedy  526019.0  
3                                      Drama  418533.0  
4                       Crime Thriller Drama  634904.0  
..                                       ...       ...  
270             Family Action Fantasy Comedy  615677.0  
271                  Drama Western Adventure  581032.0  
272                                    Drama  661914.0  
273                     Thriller Crime Drama  582014.0  
274                                    Drama  641662.0  

[275 rows x 4 columns]

In [11]:
def get_director(x):
    if " (director)" in x:
        return x.split("(director)")[0].split(";")[-1]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    elif " (director/screenplay)" in x:
        return x.split("(director/screenplay)")[0]
    else:
        return x.split("(director/​screenplay)")[0]

In [12]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(x))

In [13]:
t=0
for i in df_2020['director_name']:
    t=t+1
    print(t,i)

1 Nicolas Pesce 
2 William Eubank 
3 Miguel Arteta 
4 Jon Avnet 
5 Anthony Jerjen 
6 Andrew Desmond 
7 Daniel Farrands 
8 Ali Zamani 
9 Adil El Arbi and Bilall Fallah
10 Stephen Gaghan 
11 Tyler Perry 
12 Guy Ritchie 
13 Floria Sigismondi 
14 Todd Robinson 
15 Will Forbes 
16 Reed Morano 
17 Oz Perkins 
18 Kitty Green 
19 Cathy Yan 
20 Veronika Franz, Severin Fiala (directors/screenplay); Sergio Casci (screenplay); Riley Keough, Jaeden Martell, Lia McHugh, Richard Armitage, Alicia Silverstone
21 Tom McCarthy 
22 Jeff Baena 
23 Michael Fimognari 
24 Jeff Fowler 
25 Jeff Wadlow 
26 Stella Meghie 
27 Nat Faxon, Jim Rash
28 Drew Mylrea 
29 Lone Scherfig 
30 Chris Sanders 
31 Autumn de Wilde 
32 Dee Rees 
33 William Brent Bell 
34 Michael Cristofer 
35 Chris Henchy 
36 Leigh Whannell 
37 Brett Haley 
38 Benh Zeitlin 
39 Jason Lei Howden 
40 Dan Scanlon 
41 Gavin O'Connor 
42 Peter Berg 
43 George Nolfi 
44 Giuseppe Capotondi 
45 Carlo Mirabella-Davis 
46 Kelly Reichardt 
47 Dave S. F. Wilso

In [14]:
i=202
df_2020['Cast and crew'][i]

'Steve Byrne (director/screenplay); Jimmy O. Yang, Alex Moffat, Cedric the Entertainer'

In [15]:
def get_actor1(x):
    if " (screenplay);" in x:
        return ((x.split("(screenplay); ")[-1]).split(", ")[0]) 
    elif " (director/screenplay);"in x:
        return ((x.split("(director/screenplay); ")[-1]).split(", ")[0]) 
    elif "(screenplay),"in x:
        return ((x.split("(screenplay),")[-1]).split(", ")[0]) 
    else:
        return ((x.split('(director); ')[-1]).split(",")[0])

In [16]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(x))

In [17]:
t=0
for i in df_2020['actor_1_name']:
    t=t+1
    print(t-1,i)

0 Andrea Riseborough
1 Kristen Stewart
2 Tiffany Haddish
3 Richard Gere
4 Josh Hartnett
5 Freya Tingley
6 Mena Suvari
7 Nicola Posener
8 Will Smith
9 Robert Downey Jr.
10 Crystal R. Fox
11 Matthew McConaughey
12 Mackenzie Davis
13 Sebastian Stan
14 Terry Crews
15 Blake Lively
16 Sophia Lillis
17 Julia Garner
18 Margot Robbie
19 Riley Keough
20 Winslow Fegley
21 Alison Brie
22 Lana Condor
23 James Marsden
24 Michael Peña
25 Issa Rae
26 Julia Louis-Dreyfus
27 Drew Van Acker
28 Andrea Riseborough
29 Harrison Ford
30 Anya Taylor-Joy
31 Anne Hathaway
32 Katie Holmes
33 Tye Sheridan
34 Brian Quinn
35 Elisabeth Moss
36 Elle Fanning
37 Tommie Lynn Milazzo
38 Daniel Radcliffe
39 Tom Holland
40 Ben Affleck
41 Mark Wahlberg
42 Anthony Mackie
43 Claes Bang
44 Haley Bennett
45 John Magaro
46 Vin Diesel
47 Betty Gilpin
48 KJ Apa
49 Amy Ryan
50 Grace VanderWaal
51 Javier Bardem
52 Sidney Flanigan
53 Pete Davidson
54 Simon Pegg
55 No films had been released on this date as resulting the closure of the

In [18]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [19]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(x)).astype("str")

In [20]:
t=0
for i in df_2020['actor_2_name']:
    t=t+1
    print(t-1,i)

0 Demián Bichir
1 Vincent Cassel
2 Rose Byrne
3 Peter Dinklage
4 Margarita Levieva
5 Simon Abkarian
6 Nick Stahl
7 Houston Rhines
8 Martin Lawrence
9 Antonio Banderas
10 Phylicia Rashad
11 Charlie Hunnam
12 Finn Wolfhard
13 Christopher Plummer
14 Ludacris
15 Jude Law
16 Sammy Leakey
17 Matthew Macfadyen
18 Mary Elizabeth Winstead
19 Jaeden Martell
20 Ophelia Lovibond
21 Debby Ryan
22 Noah Centineo
23 Ben Schwartz
24 Maggie Q
25 Lakeith Stanfield
26 Will Ferrell
27 Poppy Delevingne
28 Tahar Rahim
29 Omar Sy
30 Johnny Flynn
31 Ben Affleck
32 Owain Yeoman
33 Ana de Armas
34 Joe Gatto
35 Aldis Hodge
36 Justice Smith
37 Shay Walker
38 Samara Weaving
39 Chris Pratt
40 Al Madrigal
41 Winston Duke
42 Nicholas Hoult
43 Elizabeth Debicki
44 Austin Stowell
45 Orion Hall
46 Eiza González
47 Ike Barinholtz
48 Britt Robertson
49 Thomasin McKenzie
50 Graham Verchere
51 Elle Fanning
52 Talia Ryder
53 Griffin Gluck
54 Juno Temple
55 nan
56 Ed Harris
57 Courtney B. Vance
58 Jesse Eisenberg
59 Taraji P. 

In [21]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [22]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(x))

In [23]:
t=0
for i in df_2020['actor_3_name']:
    t=t+1
    print(t-1,i)

0 John Cho
1 Jessica Henwick
2 Salma Hayek
3 Walton Goggins
4 Chandler Riggs
5 Rutger Hauer
6 Taryn Manning
7 Michael Teh
8 Vanessa Hudgens
9 Michael Sheen
10 Bresha Webb
11 Henry Golding
12 Brooklynn Prince
13 William Hurt
14 Jamila Velazquez
15 Sterling K. Brown
16 Charles Babalola
17 nan
18 Jurnee Smollett-Bell
19 Lia McHugh
20 Craig Robinson
21 John Paul Reynolds
22 Jordan Fisher
23 Tika Sumpter
24 Lucy Hale
25 Chelsea Peretti
26 Miranda Otto
27 Natasha Bassett
28 Zoe Kazan
29 Cara Gee
30 Josh O'Connor
31 Rosie Perez
32 Christopher Convery
33 Helen Hunt
34 Sal Vulcano
35 Storm Reid
36 Alexandra Shipp
37 nan
38 Natasha Liu Bordizzo
39 Julia Louis-Dreyfus
40 Michaela Watkins
41 Alan Arkin
42 Nia Long
43 Mick Jagger
44 Elizabeth Marvel
45 René Auberjonois
46 Sam Heughan
47 Amy Madigan
48 Shania Twain
49 Lola Kirke
50 Giancarlo Esposito
51 Laura Linney
52 Theodore Pellerin
53 Jon Cryer
54 Alexandra Daddario
55 nan
56 Clémence Poésy
57 Niecy Nash
58 nan
59 Betty Gilpin
60 Rachel Bloom
6

In [24]:
df_2020

Title                                      Cast and crew  \
0               The Grudge  Nicolas Pesce (director/screenplay); Andrea Ri...   
1               Underwater  William Eubank (director); Brian Duffield, Ada...   
2              Like a Boss  Miguel Arteta (director); Sam Pitman, Adam Col...   
3            Three Christs  Jon Avnet (director/screenplay); Eric Nazarian...   
4        Inherit the Viper  Anthony Jerjen (director); Andrew Crabtree (sc...   
..                     ...                                                ...   
270       We Can Be Heroes  Robert Rodriguez (director/screenplay); Priyan...   
271      News of the World  Paul Greengrass (director/screenplay); Luke Da...   
272  One Night in Miami...  Regina King (director); Kemp Powers (screenpla...   
273  Promising Young Woman  Emerald Fennell (director/screenplay); Carey M...   
274      Pieces of a Woman  Kornél Mundruczó (director); Kata Wéber (scree...   

                                      genres        id      director_name  \
0                    Horror Mystery Thriller    1970.0     Nicolas Pesce    
1    Horror Science Fiction Action Adventure  443791.0    William Eubank    
2                                     Comedy  526019.0     Miguel Arteta    
3                                      Drama  418533.0         Jon Avnet    
4                       Crime Thriller Drama  634904.0    Anthony Jerjen    
..                                       ...       ...                ...   
270             Family Action Fantasy Comedy  615677.0  Robert Rodriguez    
271                  Drama Western Adventure  581032.0   Paul Greengrass    
272                                    Drama  661914.0       Regina King    
273                     Thriller Crime Drama  582014.0   Emerald Fennell    
274                                    Drama  641662.0  Kornél Mundruczó    

              actor_1_name       actor_2_name     actor_3_name  
0       Andrea Riseborough      Demián Bichir         John Cho  
1          Kristen Stewart     Vincent Cassel  Jessica Henwick  
2          Tiffany Haddish         Rose Byrne      Salma Hayek  
3             Richard Gere     Peter Dinklage   Walton Goggins  
4            Josh Hartnett  Margarita Levieva   Chandler Riggs  
..                     ...                ...              ...  
270  Priyanka Chopra Jonas       Pedro Pascal    YaYa Gosselin  
271              Tom Hanks      Helena Zengel              NaN  
272      Kingsley Ben-Adir          Eli Goree      Aldis Hodge  
273         Carey Mulligan         Bo Burnham      Alison Brie  
274          Vanessa Kirby       Shia LaBeouf     Molly Parker  

[275 rows x 8 columns]

In [25]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [26]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title','id']]

In [27]:
new_df20

director_name           actor_1_name       actor_2_name  \
0       Nicolas Pesce      Andrea Riseborough      Demián Bichir   
1      William Eubank         Kristen Stewart     Vincent Cassel   
2       Miguel Arteta         Tiffany Haddish         Rose Byrne   
3           Jon Avnet            Richard Gere     Peter Dinklage   
4      Anthony Jerjen           Josh Hartnett  Margarita Levieva   
..                 ...                    ...                ...   
270  Robert Rodriguez   Priyanka Chopra Jonas       Pedro Pascal   
271   Paul Greengrass               Tom Hanks      Helena Zengel   
272       Regina King       Kingsley Ben-Adir          Eli Goree   
273   Emerald Fennell          Carey Mulligan         Bo Burnham   
274  Kornél Mundruczó           Vanessa Kirby       Shia LaBeouf   

        actor_3_name                                   genres  \
0           John Cho                  Horror Mystery Thriller   
1    Jessica Henwick  Horror Science Fiction Action Adventure   
2        Salma Hayek                                   Comedy   
3     Walton Goggins                                    Drama   
4     Chandler Riggs                     Crime Thriller Drama   
..               ...                                      ...   
270    YaYa Gosselin             Family Action Fantasy Comedy   
271              NaN                  Drama Western Adventure   
272      Aldis Hodge                                    Drama   
273      Alison Brie                     Thriller Crime Drama   
274     Molly Parker                                    Drama   

               movie_title        id  
0               The Grudge    1970.0  
1               Underwater  443791.0  
2              Like a Boss  526019.0  
3            Three Christs  418533.0  
4        Inherit the Viper  634904.0  
..                     ...       ...  
270       We Can Be Heroes  615677.0  
271      News of the World  581032.0  
272  One Night in Miami...  661914.0  
273  Promising Young Woman  582014.0  
274      Pieces of a Woman  641662.0  

[275 rows x 7 columns]

In [28]:
new_df20.isnull().sum()

director_name     0
actor_1_name      0
actor_2_name      0
actor_3_name     28
genres            2
movie_title       0
id                2
dtype: int64

In [29]:
new_df20=new_df20.dropna(how='any')

In [30]:
new_df20['director_name']=new_df20['director_name'].apply(lambda x:x.replace(" ",""))
new_df20['actor_1_name']=new_df20['actor_1_name'].apply(lambda x:x.replace(" ",""))
new_df20['actor_2_name']=new_df20['actor_2_name'].apply(lambda x:x.replace(" ",""))
new_df20['actor_3_name']=new_df20['actor_3_name'].apply(lambda x:x.replace(" ",""))

In [31]:
new_df20['tag'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']
new_df20['tag']=new_df20['tag'].apply(lambda x:x.lower())

In [32]:
new_df20=new_df20[["id","movie_title","tag"]]
new_df20['id']=new_df20['id'].astype("int")
new_df20

id            movie_title  \
0      1970             The Grudge   
1    443791             Underwater   
2    526019            Like a Boss   
3    418533          Three Christs   
4    634904      Inherit the Viper   
..      ...                    ...   
269  508442                   Soul   
270  615677       We Can Be Heroes   
272  661914  One Night in Miami...   
273  582014  Promising Young Woman   
274  641662      Pieces of a Woman   

                                                   tag  
0    andreariseborough demiánbichir johncho nicolas...  
1    kristenstewart vincentcassel jessicahenwick wi...  
2    tiffanyhaddish rosebyrne salmahayek miguelarte...  
3    richardgere peterdinklage waltongoggins jonavn...  
4    joshhartnett margaritalevieva chandlerriggs an...  
..                                                 ...  
269  jamiefoxx tinafey grahamnorton petedocter anim...  
270  priyankachoprajonas pedropascal yayagosselin r...  
272  kingsleyben-adir eligoree aldishodge reginakin...  
273  careymulligan boburnham alisonbrie emeraldfenn...  
274  vanessakirby shialabeouf mollyparker kornélmun...  

[246 rows x 3 columns]

## Extracting features of 2021 movies from Wikipedia

In [33]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [34]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [35]:
df

Opening  Opening.1                            Title  \
0      J A N U A R Y        1.0              Shadow in the Cloud   
1      J A N U A R Y        5.0                          Hacksaw   
2      J A N U A R Y       12.0  Dr. Bird's Advice for Sad Poets   
3      J A N U A R Y       13.0                  The White Tiger   
4      J A N U A R Y       14.0                      Locked Down   
..               ...        ...                              ...   
360  D E C E M B E R       25.0           The Tragedy of Macbeth   
361  D E C E M B E R       25.0             A Journal for Jordan   
362  D E C E M B E R       25.0                American Underdog   
363  D E C E M B E R       26.0                          Memoria   
364  D E C E M B E R        NaN                              NaN   

                                    Production company  \
0                               Vertical Entertainment   
1                     Leone Films / Midnight Releasing   
2             Relativity Media / Ketchup Entertainment   
3             Netflix / ARRAY / Purple Pebble Pictures   
4                      HBO Max / Warner Bros. Pictures   
..                                                 ...   
360                        Apple TV+ / A24 / IAC Films   
361  Columbia Pictures / Escape Artists / Bron Studios   
362                                          Lionsgate   
363                                               Neon   
364                                                NaN   

                                         Cast and crew   Ref.  
0    Roseanne Liang (director/screenplay); Max Land...    [2]  
1    Anthony Leone (director/screenplay); Amy Cay, ...    [3]  
2    Yaniv Raz (director/screenplay); Lucas Jade Zu...    [4]  
3    Ramin Bahrani (director/screenplay); Adarsh Go...    NaN  
4    Doug Liman (director); Steven Knight (screenpl...    [5]  
..                                                 ...    ...  
360  Joel Coen (director/screenplay); Denzel Washin...  [277]  
361  Denzel Washington (director); Virgil Williams ...  [278]  
362  Erwin brothers (directors); Jon Erwin, David A...  [279]  
363  Apichatpong Weerasethakul (director/acreenplay...  [280]  
364                                                NaN    NaN  

[365 rows x 6 columns]

In [36]:
df['genres'] = df.progress_apply(lambda row: safe_get_genre(str(row['Title']), row.name), axis=1)

 68%|██████████████████████████████████████████████████████▏                         | 247/365 [06:27<02:42,  1.38s/it]

Error processing index: 245
Title: Dating and New York
Error: attribute name must be string, not 'int'


100%|████████████████████████████████████████████████████████████████████████████████| 365/365 [09:34<00:00,  1.58s/it]


In [37]:
df['id'] = df.progress_apply(lambda row: safe_get_id(str(row['Title']), row.name), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████| 365/365 [00:00<00:00, 3953.89it/s]

Error processing index: 245
Title: Dating and New York
Error: attribute name must be string, not 'int'


In [38]:
df_2021 = df[['Title','Cast and crew','genres','id']]

In [39]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    elif " (director/acreenplay);" in x:
        return x.split("(director/acreenplay); ")[0]
    elif "(director/screenwriter);" in x:
        return x.split("(director/screenwriter);")[0]
    elif "(directors/screenplay);" in x:
        return x.split("(directors/screenplay);")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [40]:
df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))

In [41]:
t=0
for i in df_2021['director_name']:
    t=t+1
    print(t-1,i)

0 Roseanne Liang
1 Anthony Leone
2 Yaniv Raz
3 Ramin Bahrani
4 Doug Liman
5 Simon Stone
6 Mikael Håfström
7 Robert Lorenz
8 Bennett Lasseter
9 Nate Parker
10 Alex McAulay
11 Gabriela Cowperthwaite
12 Conor Allyn
13 Alex Ranarivelo
14 Mike P. Nelson
15 Glenda Ivin
16 John Lee Hancock
17 Jude Weng
18 Chloé Zhao
19 Sam Levinson
20 Fisher Stevens
21 Viggo Mortensen
22 Mike Cahill
23 Chad Hartigan
24 Barry Alexander Brown
25 Ben Hozie
26 Sia
27 Shaka King
28 Kevin Macdonald
29 Tate Taylor
30 Josh Greenbaum
31 Lee Isaac Chung
32 Michael Fimognari
33 Mona Fastvold
34 Azazel Jacobs
35 Robin Wright
36 Kevin Lewis
37 Ian Samuels
38 Castille Landon
39 J Blakeson
40 Lena Khan
41 Tiller Russell
42 Tim Story
43 Lee Daniels
44 Anthony and Joe Russo
45 Nicholas Jarecki
46 R. J. Cutler
47 Keith Thomas
48 Amy Poehler
49 Craig Brewer
50 Tim Hill
51 Don Hall, Carlos López Estrada
52 Doug Liman
53 Joe Carnahan
54 Eddie Huang
55 Miguel Arteta
56 Steve Basilone
57 Edward Drake
58 Brian DeCubellis
59 Zack Sny

In [42]:
def get_actor1(x):
    if " (screenplay)" in x:
         if len(x.split("(screenplay); "))>1:
            return ((x.split("(screenplay); ")[-1]).split(", ")[0]) 
         else:
            return np.NaN
        
    elif " (screenwriter)" in x:
        return ((x.split('(screenwriter); ')[-1]).split(",")[0]) 
    
    elif " (director/screenwriter)" in x:
        return ((x.split('(director/screenwriter); ')[-1]).split(",")[0]) 
    
    elif " (co-director/screenplay)" in x:
        return ((x.split('(co-director/screenplay); ')[-1]).split(",")[0])
    
    elif " (director/screenplay)" in x :
        return ((x.split("(director/screenplay); ")[-1]).split(", ")[0]) 
    
    elif " (director)"in x:
        if len(x.split('(director); '))>1:
            return ((x.split('(director); ')[-1]).split(",")[0])
        else:
            return np.NaN

        
    elif " (director/acreenplay)"in x:
        return ((x.split('(director/acreenplay); ')[-1]).split(",")[0]) 
    
    else:
        return ((x.split("(director/screenplay); ")[-1]).split(", ")[0])

In [43]:
df_2021['actor_1_name'] = df_2021['Cast and crew'].astype(str).map(lambda x: get_actor1(x)).astype("str")

In [44]:
t=0
for i in df_2021['actor_1_name']:
    t=t+1
    print(t,i)

1 Chloë Grace Moretz
2 Amy Cay
3 Lucas Jade Zumann
4 Adarsh Gourav
5 Anne Hathaway
6 Carey Mulligan
7 Anthony Mackie
8 Liam Neeson
9 Keean Johnson
10 Nate Parker
11 Jack Dylan Grazer
12 Jason Segel
13 Jake Allyn
14 Sean Patrick Flanery
15 Charlotte Vega
16 Naomi Watts
17 Denzel Washington
18 Kea Peahu
19 Frances McDormand
20 Zendaya
21 Justin Timberlake
22 Viggo Mortensen
23 Owen Wilson
24 Olivia Cooke
25 Lucas Till
26 Julia Fox
27 Kate Hudson
28 Daniel Kaluuya
29 Jodie Foster
30 Allison Janney
31 Kristen Wiig
32 Steven Yeun
33 Lana Condor
34 Katherine Waterston
35 Michelle Pfeiffer
36 Robin Wright
37 Nicolas Cage
38 Kathryn Newton
39 Katherine Heigl
40 Rosamund Pike
41 Matilda Lawler
42 Jason Clarke
43 Chloë Grace Moretz
44 Andra Day
45 Tom Holland
46 Gary Oldman
47 Billie Eilish
48 Dave Davis
49 Hadley Robinson
50 Eddie Murphy
51 Tom Kenny
52 Kelly Marie Tran
53 Daisy Ridley
54 Frank Grillo
55 Taylor Takahashi
56 Jennifer Garner
57 Finn Wittrock
58 Bruce Willis
59 Victoria Justice
60

In [45]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    
    elif len((x.split("(directors); ")[-1]).split(", ")) < 2:
        return np.NaN
    
    elif " (screenplay)," in x:
        return ((x.split("(screenplay), ")[-1]).split(", ")[1])
    
    elif " (screenplay);" in x:
        return ((x.split("(screenplay); ")[-1]).split(", ")[1])
    
    elif " (directors/screenplay)" in x:
        return ((x.split("(directors/screenplay); ")[-1]).split(", ")[1])
    
    elif " (director/screenplay)" in x: 
        return ((x.split("(director/screenplay); ")[-1]).split(", ")[1])
    else:
        return ((x.split("(director); ")[-1]).split(", ")[1])

In [46]:
df_2021['actor_2_name'] = df_2021['Cast and crew'].astype(str).map(lambda x: get_actor2(x)).astype("str")

In [47]:
t=0
for i in df_2021['actor_2_name']:
    t=t+1
    print(t,i)

1 Taylor John Smith
2 Brian Patrick Butler
3 Taylor Russell
4 Rajkummar Rao
5 Chiwetel Ejiofor
6 Ralph Fiennes
7 Damson Idris
8 Katheryn Winnick
9 Madeline Brewer
10 Omari Hardwick
11 Fionn Whitehead
12 Dakota Johnson
13 Frank Grillo
14 Dennis Quaid
15 Adain Bradley
16 Andrew Lincoln
17 Rami Malek
18 Alex Aiono
19 David Strathairn
20 John David Washington
21 Juno Temple
22 Lance Henriksen
23 Salma Hayek
24 Jack O'Connell
25 Lucy Hale
26 Nikki Belfiglio
27 Leslie Odom Jr.
28 Lakeith Stanfield
29 Tahar Rahim
30 Mila Kunis
31 Annie Mumolo
32 Han Ye-ri
33 Noah Centineo
34 Vanessa Kirby
35 Lucas Hedges
36 Demián Bichir
37 Emily Tosta
38 Kyle Allen
39 Madison Iseman
40 Peter Dinklage
41 Alyson Hannigan
42 Nick Robinson
43 Michael Peña
44 Trevante Rhodes
45 Ciara Bravo
46 Armie Hammer
47 Finneas O'Connell
48 Menashe Lustig
49 Lauren Tsai
50 Arsenio Hall
51 Awkwafina
52 Awkwafina
53 Tom Holland
54 Mel Gibson
55 Taylour Paige
56 Édgar Ramírez
57 Zoë Chao
58 Frank Grillo
59 Matthew Daddario
60 H

In [48]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [49]:
df_2021['actor_3_name'] = df_2021['Cast and crew'].astype(str).map(lambda x: get_actor3(x)).astype("str")

In [50]:
t=0
for i in df_2021['actor_3_name']:
    t=t+1
    print(t,i)

1 Beulah Koale
2 Michael C. Burgess
3 Chase Stokes
4 Priyanka Chopra Jonas
5 Stephen Merchant
6 Lily James
7 Emily Beecham
8 Juan Pablo Raba
9 Rya Kihlstedt
10 Theo Rossi
11 Rainn Wilson
12 Casey Affleck
13 Jorge A. Jiménez
14 Katrina Bowden
15 Bill Sage
16 Jacki Weaver
17 Jared Leto
18 Lindsay Watson
19 Linda May
20 nan
21 Alisha Wainwright
22 Terry Chen
23 nan
24 Raúl Castillo
25 Cedric the Entertainer
26 Austin Brown
27 Maddie Ziegler
28 Jesse Plemons
29 Shailene Woodley
30 Awkwafina
31 Jamie Dornan
32 Alan Kim
33 Jordan Fisher
34 Christopher Abbott
35 Tracy Letts
36 Kim Dickens
37 Ric Reitz
38 Al Madrigal
39 Israel Broussard
40 Eiza González
41 Ben Schwartz
42 Katie Aselton
43 Colin Jost
44 Garrett Hedlund
45 Jack Reynor
46 Evangeline Lilly
47 Maggie Baird
48 Malky Goldman
49 Patrick Schwarzenegger
50 Jermaine Fowler
51 Matt Berry
52 Izaac Wang
53 Mads Mikkelsen
54 Naomi Watts
55 Jorge Lendeborg Jr.
56 Jenna Ortega
57 Casey Wilson
58 Brandon Thomas Lee
59 Katherine McNamara
60 Amy 

In [51]:
df_2021 = df_2021.rename(columns={'Title':'movie_title'})

In [52]:
df_2021

movie_title  \
0                Shadow in the Cloud   
1                            Hacksaw   
2    Dr. Bird's Advice for Sad Poets   
3                    The White Tiger   
4                        Locked Down   
..                               ...   
360           The Tragedy of Macbeth   
361             A Journal for Jordan   
362                American Underdog   
363                          Memoria   
364                              NaN   

                                         Cast and crew  \
0    Roseanne Liang (director/screenplay); Max Land...   
1    Anthony Leone (director/screenplay); Amy Cay, ...   
2    Yaniv Raz (director/screenplay); Lucas Jade Zu...   
3    Ramin Bahrani (director/screenplay); Adarsh Go...   
4    Doug Liman (director); Steven Knight (screenpl...   
..                                                 ...   
360  Joel Coen (director/screenplay); Denzel Washin...   
361  Denzel Washington (director); Virgil Williams ...   
362  Erwin brothers (directors); Jon Erwin, David A...   
363  Apichatpong Weerasethakul (director/acreenplay...   
364                                                NaN   

                                genres        id               director_name  \
0                    Horror Action War  675327.0              Roseanne Liang   
1    Adventure Family TV Movie Western  275506.0               Anthony Leone   
2                         Comedy Drama  533705.0                   Yaniv Raz   
3                                Drama  628534.0               Ramin Bahrani   
4                 Comedy Crime Romance  741228.0                  Doug Liman   
..                                 ...       ...                         ...   
360                          Drama War  591538.0                   Joel Coen   
361                      Drama Romance  501841.0           Denzel Washington   
362                              Drama  673309.0              Erwin brothers   
363      Drama Science Fiction Mystery  511819.0  Apichatpong Weerasethakul    
364                        Drama Crime  384717.0                         nan   

           actor_1_name          actor_2_name           actor_3_name  
0    Chloë Grace Moretz     Taylor John Smith           Beulah Koale  
1               Amy Cay  Brian Patrick Butler     Michael C. Burgess  
2     Lucas Jade Zumann        Taylor Russell           Chase Stokes  
3         Adarsh Gourav         Rajkummar Rao  Priyanka Chopra Jonas  
4         Anne Hathaway      Chiwetel Ejiofor       Stephen Merchant  
..                  ...                   ...                    ...  
360   Denzel Washington     Frances McDormand          Bertie Carvel  
361   Michael B. Jordan          Chanté Adams        Jalon Christian  
362        Zachary Levi           Anna Paquin           Dennis Quaid  
363       Tilda Swinton            Elkin Díaz         Jeanne Balibar  
364                 nan                   nan                    nan  

[365 rows x 8 columns]

In [53]:
new_df21 = df_2021.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title','id']]

In [54]:
new_df21['director_name']=new_df21['director_name'].apply(lambda x:x.replace(" ",""))
new_df21['actor_1_name']=new_df21['actor_1_name'].apply(lambda x:x.replace(" ",""))
new_df21['actor_2_name']=new_df21['actor_2_name'].apply(lambda x:x.replace(" ",""))
new_df21['actor_3_name']=new_df21['actor_3_name'].apply(lambda x:x.replace(" ",""))

In [55]:
new_df21 = new_df21.dropna(how='any')

In [56]:
new_df21['tag'] = new_df21['actor_1_name'] + ' ' + new_df21['actor_2_name'] + ' '+ new_df21['actor_3_name'] + ' '+ new_df21['director_name'] +' ' + new_df21['genres']
new_df21['tag']=new_df21['tag'].str.lower()
new_df21['id']=new_df21['id'].astype("int")

In [57]:
new_df21=new_df21[["id","movie_title","tag"]]
new_df21

id                      movie_title  \
0    675327              Shadow in the Cloud   
1    275506                          Hacksaw   
2    533705  Dr. Bird's Advice for Sad Poets   
3    628534                  The White Tiger   
4    741228                      Locked Down   
..      ...                              ...   
359  476669                   The King's Man   
360  591538           The Tragedy of Macbeth   
361  501841             A Journal for Jordan   
362  673309                American Underdog   
363  511819                          Memoria   

                                                   tag  
0    chloëgracemoretz taylorjohnsmith beulahkoale r...  
1    amycay brianpatrickbutler michaelc.burgess ant...  
2    lucasjadezumann taylorrussell chasestokes yani...  
3    adarshgourav rajkummarrao priyankachoprajonas ...  
4    annehathaway chiwetelejiofor stephenmerchant d...  
..                                                 ...  
359  ralphfiennes gemmaarterton rhysifans matthewva...  
360  denzelwashington francesmcdormand bertiecarvel...  
361  michaelb.jordan chantéadams jalonchristian den...  
362  zacharylevi annapaquin dennisquaid erwinbrothe...  
363  tildaswinton elkindíaz jeannebalibar apichatpo...  

[363 rows x 3 columns]

In [58]:
my_df = pd.concat([new_df20, new_df21], ignore_index=True)

In [59]:
my_df

id             movie_title  \
0      1970              The Grudge   
1    443791              Underwater   
2    526019             Like a Boss   
3    418533           Three Christs   
4    634904       Inherit the Viper   
..      ...                     ...   
604  476669          The King's Man   
605  591538  The Tragedy of Macbeth   
606  501841    A Journal for Jordan   
607  673309       American Underdog   
608  511819                 Memoria   

                                                   tag  
0    andreariseborough demiánbichir johncho nicolas...  
1    kristenstewart vincentcassel jessicahenwick wi...  
2    tiffanyhaddish rosebyrne salmahayek miguelarte...  
3    richardgere peterdinklage waltongoggins jonavn...  
4    joshhartnett margaritalevieva chandlerriggs an...  
..                                                 ...  
604  ralphfiennes gemmaarterton rhysifans matthewva...  
605  denzelwashington francesmcdormand bertiecarvel...  
606  michaelb.jordan chantéadams jalonchristian den...  
607  zacharylevi annapaquin dennisquaid erwinbrothe...  
608  tildaswinton elkindíaz jeannebalibar apichatpo...  

[609 rows x 3 columns]

In [60]:
my_df.isnull().sum()

id             0
movie_title    0
tag            0
dtype: int64

In [61]:
my_df.shape

(609, 3)

In [62]:
old_df = pd.read_csv('till 2019.csv')

In [63]:
old_df

movie_title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
5591                         Spies in Disguise   
5592                              Little Women   
5593                                      1917   
5594                                Just Mercy   
5595                                  Clemency   

                                                    tag        id  
0     cchpounder joeldavidmoore wesstudi jamescamero...   19995.0  
1     johnnydepp orlandobloom jackdavenport goreverb...     285.0  
2     christophwaltz rorykinnear stephaniesigman sam...  206647.0  
3     tomhardy christianbale josephgordon-levitt chr...   49026.0  
4     darylsabara samanthamorton pollywalker andrews...   49529.0  
...                                                 ...       ...  
5591  willsmith tomholland rashidajones nickbruno,tr...  431693.0  
5592  saoirseronan emmawatson florencepugh gretagerw...  331482.0  
5593  georgemackay dean-charleschapman markstrong sa...  530915.0  
5594  michaelb.jordan jamiefoxx brielarson destindan...  522212.0  
5595  alfrewoodard wendellpierce aldishodge chinonye...  565307.0  

[5596 rows x 3 columns]

In [64]:
final_df = pd.concat([old_df, my_df], ignore_index=True)

In [65]:
final_df

movie_title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
6200                            The King's Man   
6201                    The Tragedy of Macbeth   
6202                      A Journal for Jordan   
6203                         American Underdog   
6204                                   Memoria   

                                                    tag        id  
0     cchpounder joeldavidmoore wesstudi jamescamero...   19995.0  
1     johnnydepp orlandobloom jackdavenport goreverb...     285.0  
2     christophwaltz rorykinnear stephaniesigman sam...  206647.0  
3     tomhardy christianbale josephgordon-levitt chr...   49026.0  
4     darylsabara samanthamorton pollywalker andrews...   49529.0  
...                                                 ...       ...  
6200  ralphfiennes gemmaarterton rhysifans matthewva...  476669.0  
6201  denzelwashington francesmcdormand bertiecarvel...  591538.0  
6202  michaelb.jordan chantéadams jalonchristian den...  501841.0  
6203  zacharylevi annapaquin dennisquaid erwinbrothe...  673309.0  
6204  tildaswinton elkindíaz jeannebalibar apichatpo...  511819.0  

[6205 rows x 3 columns]

In [66]:
final_df.isnull().sum()

movie_title    0
tag            0
id             0
dtype: int64

In [67]:
final_df.to_csv('till 2021.csv',index=False)